### tính toán và vẽ biểu đồ tròn của trang bị và tộc hệ

In [9]:
import pandas as pd
import numpy as np
import ast
from itertools import chain

#### đọc dữ liệu để lấy món đồ

In [10]:
item_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['List Name Item']).drop_duplicates().reset_index(drop=True)

In [11]:
list_items = []
for i in range(len(item_df['List Name Item'])):
    list_items.append(ast.literal_eval(item_df['List Name Item'][i])) 

### TÌM RA MÓN ĐỒ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

hàm zip dùng để biến 2 list được cho ra thành key và values

lisst(chain) dùng để ép mảng xuống còn 1D và sau đó sort lại

In [12]:
def get_best_worst_items(list_items):
    flatten_list = list(chain.from_iterable(list_items))
    flatten_list = list(chain.from_iterable(flatten_list))
    flatten_list = np.array(flatten_list, dtype = object)
    unique, counts = np.unique(flatten_list, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_item = list(b.items())
    worst_item = list(b.items())[0:5]
    return best_item,worst_item

In [13]:
get_best_worst_items(list_items)

([('TFT5_Item_MorellonomiconRadiant', 12),
  ('TFT5_Item_ShroudOfStillnessRadiant', 12),
  ('TFT5_Item_FrozenHeartRadiant', 14),
  ('TFT5_Item_LastWhisperRadiant', 14),
  ('TFT5_Item_RedemptionRadiant', 14),
  ('TFT5_Item_BloodthirsterRadiant', 15),
  ('TFT5_Item_RunaansHurricaneRadiant', 16),
  ('TFT5_Item_HextechGunbladeRadiant', 19),
  ('TFT5_Item_InfinityEdgeRadiant', 19),
  ('TFT5_Item_RapidFirecannonRadiant', 20),
  ('TFT5_Item_ChaliceOfPowerRadiant', 22),
  ('TFT5_Item_DeathbladeRadiant', 22),
  ('TFT5_Item_GargoyleStoneplateRadiant', 24),
  ('TFT5_Item_LocketOfTheIronSolariRadiant', 24),
  ('TFT5_Item_QuicksilverRadiant', 24),
  ('TFT5_Item_StatikkShivRadiant', 24),
  ('TFT5_Item_BrambleVestRadiant', 25),
  ('TFT5_Item_GuardianAngelRadiant', 25),
  ('TFT5_Item_SpearOfShojinRadiant', 25),
  ('TFT5_Item_ZzRotPortalRadiant', 25),
  ('TFT5_Item_ArchangelsStaffRadiant', 26),
  ('TFT5_Item_RabadonsDeathcapRadiant', 26),
  ('TFT5_Item_TrapClawRadiant', 26),
  ('TFT5_Item_BlueBuffRadia

### TÌM VỊ TRÍ TRUNG BÌNH CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU


khi người chơi sử dụng món đồ đó thì sẽ được lọt vào top mấy của game

In [14]:
average_df = pd.read_csv('process_data_tft_each_item_csv.csv', usecols = ['Puuid', 'Placement', 'List Name Item']).drop_duplicates().reset_index(drop = True)

In [15]:
def convert_string_to_list(ok, placement):
    temp = ok[ok.eq(placement).any(1)].reset_index(drop=True)
    top = []
    for i in range(len(temp)):
        top.append(ast.literal_eval(temp['List Name Item'][i]))
    return top

In [16]:
item_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(average_df, i)
    temp1 = 'top_' + str(i)
    item_on_top[temp1] = get_best_worst_items(temp)


In [17]:
sum = 0
temp = 0
# print(dict(item_on_top['top_' + str(5)][0]))
for i in range(1,9,1):
        cc = dict(item_on_top['top_' + str(i)][0])
        if('TFT5_Item_MorellonomiconRadiant' in cc):
                # print(dict(item_on_top['top_' + str(i)][0])['TFT5_Item_MorellonomiconRadiant'])
                temp = temp+dict(item_on_top['top_' + str(i)][0])['TFT5_Item_MorellonomiconRadiant']*i
                sum = sum + (dict(item_on_top['top_' + str(i)][0])['TFT5_Item_MorellonomiconRadiant'])
print(temp/sum)
print(sum/32000)

3.75
0.000375


### TƯƠNG TỰ VỚI TỘC VÀ HỆ

In [18]:
traits_df = pd.read_csv('process_data_tft_general_csv.csv', usecols = ['List Traits']).drop_duplicates().reset_index(drop=True)

In [19]:
list_traits = []

for i in range(len(traits_df['List Traits'])):
    list_traits.append(ast.literal_eval(traits_df['List Traits'][i])) 

In [20]:
def get_best_worst_traits(list_traits):
    flatten_list = list(chain.from_iterable(list_traits))
    flatten_list = np.array(flatten_list, dtype = object)
    cc = flatten_list[0::3]
    unique, counts= np.unique(cc, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_traits = list(b.items())[-5:]
    worst_traits = list(b.items())[:5]
    return best_traits, worst_traits

In [21]:
a, b = get_best_worst_traits(list_traits)
print(a, b)

[('Set8_Brawler', 8371), ('Set8_OxForce', 8558), ('Set8_Mascot', 8655), ('Set8_StarGuardian', 9310), ('Set8_Aegis', 10518)] [('Set8_Arsenal', 1865), ('Set8_Corrupted', 2397), ('Set8_Forecaster', 2778), ('Set8_Supers', 3134), ('Set8_Recon', 3143)]


### top 1 và top 4

In [22]:
df4 = pd.read_csv('process_data_tft_general_csv.csv', usecols= ['Placement', 'List Traits'])

In [23]:
traits_top_1 = df4[df4.eq(1).any(1)].reset_index(drop=True)
temp = []
for i in range(len(traits_top_1['List Traits'])):
    temp.append(ast.literal_eval(traits_top_1['List Traits'][i])) 

a, b = get_best_worst_traits(temp)
print(a, b)

[('Set8_Brawler', 2556), ('Set8_Heart', 2565), ('Set8_Mascot', 2765), ('Set8_StarGuardian', 2907), ('Set8_Aegis', 3250)] [('Set8_Arsenal', 599), ('Set8_Recon', 690), ('Set8_Deadeye', 876), ('Set8_Corrupted', 908), ('Set8_Ace', 1037)]
